# Python建模库介绍（Introduction to Modeling Libraries ）

- [statsmodels](http://statsmodels.org)

- [scikit-learn](http://scikit-learn.org)

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## 1.pandas与建模代码的结合（Interfacing Between pandas and Model Code）

- `.values`属性可将pandas的DataFrame对象转换为NumPy数组

    - 一般用于同构化的数据，异构化的数据会转换成ndarray

- 创建虚拟变量连接结果以替代category列

In [5]:
import pandas as pd
import numpy as np

# 采用同构化数据，即数据类型一致
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data
data.columns
print(data.values, '\n')

df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
print(df2, '\n')

model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

[[ 1.    0.01 -1.5 ]
 [ 2.   -0.01  0.  ]
 [ 3.    0.25  3.6 ]
 [ 4.   -4.1   1.3 ]
 [ 5.    0.   -2.  ]] 

   one   two  three
0  1.0  0.01   -1.5
1  2.0 -0.01    0.0
2  3.0  0.25    3.6
3  4.0 -4.10    1.3
4  5.0  0.00   -2.0 



array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [6]:
# 采用异构化数据
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
print(data, '\n')
# 用创建的虚拟变量连接结果代替所创建的category列
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

   x0    x1    y category
0   1  0.01 -1.5        a
1   2 -0.01  0.0        b
2   3  0.25  3.6        a
3   4 -4.10  1.3        a
4   5  0.00 -2.0        b 



,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 2.使用Pasty创建模型描述（Creating Model Descriptions with Patsy）

- [Pasty](http://pasty.resdthedocs.io/) 是一个用于描述统计模型（尤其是线性模型）的Python库

    - 它的公式是字符串特殊语法： `y ~ x0 + x1`

In [12]:
import patsy

data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data
y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [11]:
# DesignMatrix是含有附加元数据的NumPy ndarray，Intercept（截距）源于线性模型
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [13]:
np.asarray(y)
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [14]:
# 通过给模型添加名词列 '+o' 以加入截距
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [16]:
# 将Pasty对象传给如下算法会执行一个最小二乘回归
coef, resid, _, _ = np.linalg.lstsq(X, y)
coef

/var/folders/v0/q2247qvj6nx2z3kxhc2_snyw0000gn/T/ipykernel_5838/3739554016.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


array([[ 0.3129],
       [-0.0791],
       [-0.2655]])

In [17]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### ①Pasty公式中的数据转换（Data Transformations in Patsy Formulas）

在封闭作用域内使用函数进行变量转换，如标准化（均值为0和方差为1）和居中（减去平均值）。

作为建模的一部分过程，可能会在一个数据集上拟合一个模型，之后基于另一个模型评价该模型。这个过程可以保留部分数据或之后加入新数据，在应用像居中和标准化这样的转换时，在基于新数据用模型预测时（通常称为有状态的转换）要小心，因为在形成新的数据集时必须使用原数据集的均值和标准差等统计值。

`pasty.build_design_matrices` 函数可以使用原始样本内的数据集中保存的信息将变换应用于新的样本外数据上

In [18]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [19]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [20]:
# 将原始样本内数据集中保存的信息将变换应用于新的样本数据上
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [21]:
# 当你想要将数据集中两列按列名相加时，必须将列明封装到特殊的I函数中
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### ②分类数据与Pasty（Categorical Data and Patsy）

In [22]:
# 在Pasty公式中使用非数字名词时，它们会被默认转换为虚拟变量
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [23]:
# 如果忽略截距，每个类别值的列将会被包含在模型的设计矩阵里
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [24]:
# 数字列可以使用C函数解释为分类类型
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [25]:
# 使用多个分类名词
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 3.statsmodels介绍

statsmodels是一个Python库，用于拟合多种统计模型，执行统计测试和数据探索可视化。包含在其中的一些模型：

- 线性模型，广义线性模型和鲁棒线性模型

- 线性混合效应模型

- 方差分析（ANOVA）方法

- 时间序列过程和状态空间模型

- 广义的矩量法

### ①评估线性模型（Estimating Linear Models）

- 基于公式的： `import statsmodels.formula.api as smf`

- 基于数组的： `import statsmodels.api as sm`

In [26]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [27]:
# 根据随机数据生成线性模型

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# 用于复现（reproducibility）
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [29]:
X[:5]
y[:5]

array([ 0.4279, -0.6735, -0.0909, -0.4895, -0.1289])

In [30]:
# 将截距添加到现有的矩阵中
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.    , -0.1295, -1.2128,  0.5042],
       [ 1.    ,  0.3029, -0.4357, -0.2542],
       [ 1.    , -0.3285, -0.0253,  0.1384],
       [ 1.    , -0.3515, -0.7196, -0.2582],
       [ 1.    ,  1.2433, -0.3738, -0.5226]])

In [34]:
# 拟合一个最小二乘回归
model = sm.OLS(y, X)
# 返回一个回归结果对象
results = model.fit()
print(results.params, type(results.params), '\n')

print(results.summary())

[0.1783 0.223  0.501 ] <class 'numpy.ndarray'> 

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                    7.44e-12
Time:                        17:25:49   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025     

In [37]:
# 假设所有模型参数都在DataFrame中
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

# 使用statsmodels公式
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
print(results.params, '\n')
print(results.tvalues, '\n')

# 根据估计的模型参数计算预测值
results.predict(data[:5])

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64 

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64 



0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### ②评估时间序列处理（Estimating Time Series Processes）

时间序列分析：

- 自回归过程

- 卡尔曼滤波

- 多变量自回归模型

In [38]:
# 模拟一些具有自回归结构和噪声的时间序列数据
# 注意：AR has been removed from statsmodels and replaced with statsmodels.tsa.ar_model.AutoReg.

init_x = 4

import random
values = [init_x, init_x]
N = 1000
# 该数据带有b1、b2两个滞后
b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [49]:
# 使用更大的滞后数来拟合该模型
# 注意：AR has been removed from statsmodels and replaced with statsmodels.tsa.ar_model.AutoReg.该代码块错误
""" from statsmodels.tsa.ar_model import AutoReg

MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS) """

' from statsmodels.tsa.ar_model import AutoReg\n\nMAXLAGS = 5\nmodel = sm.tsa.AR(values)\nresults = model.fit(MAXLAGS) '

## 4.scikit-learn介绍

包含广泛的标准监督和无监督的的机器学习方法，包含用于模型评估和选择、数据转换、数据加载的模型持久化的工具。

此节使用[Kaggle比赛中关于泰坦尼克号上生还乘客的经典数据集](https://www.kaggle.com/c/titanic)

In [51]:
# 载入测试和训练数据集
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [52]:
# statsmodels和scikit-learn通常不能提供缺失数据，需要检查下
train.isnull().sum()
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [54]:
# 根据Age特征作为乘客是否能够生还的预测依据，使用训练集的中间值填充两个表的空值
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

# 明确模型
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

# 创建NumPy数组
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
X_train[:5]
y_train[:5]

array([0, 1, 1, 1, 0])

In [59]:
# 创建一个模型实例
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# 拟合模型
model.fit(X_train, y_train)

# 预测
y_predict = model.predict(X_test)
y_predict[:10]

# 如果有测试数据集的真实值，可以计算精度百分比或其他一些错误指标
# (y_true == y_predict).mean()

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [63]:
# 使用可用于参数调整的交叉验证以避免过度拟合训练数据
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.7758, 0.7982, 0.7758, 0.7883])